In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import pandas_datareader as pdr
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

import pandas as pd
import numpy as np

import talib

In [4]:
idx = pd.IndexSlice

In [5]:
data = pd.read_hdf('us_stocks.h5', 'us_stocks')

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 11343920 entries, ('A', Timestamp('2000-01-03 00:00:00')) to ('ZUMZ', Timestamp('2018-03-27 00:00:00'))
Data columns (total 5 columns):
 #   Column  Dtype  
---  ------  -----  
 0   open    float64
 1   high    float64
 2   low     float64
 3   close   float64
 4   volume  float64
dtypes: float64(5)
memory usage: 476.4+ MB


open       high        low      close     volume
ticker date                                                             
A      2000-01-03  53.726454  53.856080  45.969377  49.121329  3343600.0
       2000-01-04  46.481058  46.992738  44.175084  45.369006  3408500.0
       2000-01-05  45.198445  45.239380  41.828176  41.998737  4119200.0
       2000-01-06  42.046493  42.298923  39.658651  40.934441  1812900.0
       2000-01-07  40.293135  44.986951  40.252200  44.345645  2016900.0

## Step 1

Select the adjusted open, high, low, and close prices, as well as the volume, for all tickers from the Quandl Wiki data that you downloaded and simplified for the last milestone for the 2006–2016 time period.

Looking ahead, we will use 2014–2016 as our ‘out-of-sample’ period to test the performance of a strategy based on a machine learning model selected using data from preceding periods. We’re including some earlier years to ensure all indicators (some of which are computed over longer windows) have data for the period of interest.

In [6]:
# Select price data for the 2006-2016 time period
data = data.loc[idx[:, '2006':'2016'], :]

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7532460 entries, ('A', Timestamp('2006-01-03 00:00:00')) to ('ZUMZ', Timestamp('2016-12-30 00:00:00'))
Data columns (total 5 columns):
 #   Column  Dtype  
---  ------  -----  
 0   open    float64
 1   high    float64
 2   low     float64
 3   close   float64
 4   volume  float64
dtypes: float64(5)
memory usage: 316.1+ MB


open       high        low      close     volume
ticker date                                                             
A      2006-01-03  22.786839  22.909642  22.387796  22.855063  3796200.0
       2006-01-04  22.889175  23.080202  22.766372  22.916465  3001300.0
       2006-01-05  22.820951  23.516836  22.820951  23.516836  3458800.0
       2006-01-06  23.537304  23.735153  23.257585  23.639640  4396500.0
       2006-01-09  23.639640  23.741976  23.482724  23.571416  2920500.0

In [7]:
print(f"# Tickers: {len(data.index.unique('ticker')):,.0f} | # Dates: {len(data.index.unique('date')):,.0f}")

# Tickers: 3,186 | # Dates: 2,846


## ~~Step 3~~ Step 2

Compute daily returns and keep only ‘inliers’ with values between –100% and +100% as a basic check against data error.

In [8]:
daily_returns = data.groupby('ticker').close.pct_change()
daily_returns.head()

ticker  date      
A       2006-01-03         NaN
        2006-01-04    0.002687
        2006-01-05    0.026198
        2006-01-06    0.005222
        2006-01-09   -0.002886
Name: close, dtype: float64

In [9]:
# Drop count using .iloc[1:]
daily_returns.describe(percentiles=[.00001, .0001, .001, .999, .9999, .99999]).iloc[1:]

mean         0.000797
std          0.139968
min         -0.993615
0.001%      -0.706667
0.01%       -0.396670
0.1%        -0.192994
50%          0.000000
99.9%        0.233070
99.99%       0.586986
99.999%      1.890708
max        283.000000
Name: close, dtype: float64

In [10]:
outliers = daily_returns[(daily_returns < daily_returns.quantile(.00001)) |
                         (daily_returns > daily_returns.quantile(.99999))]

print(f'# Observations: {len(outliers):,.0f} | # Tickers: {len(outliers.index.unique("ticker")):,.0f}')

# Observations: 151 | # Tickers: 90


In [11]:
data = data.drop(index=outliers.index.unique('ticker'), level='ticker')

print(f"# Tickers: {len(data.index.unique('ticker')):,.0f} | # Dates: {len(data.index.unique('date')):,.0f}")

# Tickers: 3,096 | # Dates: 2,799


## Step 2

Compute the dollar volume as the product of closing price and trading volume, then select the stocks with at least eight years of data and the lowest average daily rank for this metric.

We are simplifying here a little bit: in practice, you would want to identify the universe as a rolling average (e.g., on monthly or quarterly basis, depending on our train/test parameters in the next steps) of the dollar volume to avoid including information ‘from the future’ that could introduce a lookahead bias. Feel free to select the universe in this methodologically more robust yet computationally a bit more intensive way.

In [12]:
dv = data.close.mul(data.volume)
dv.head()

ticker  date      
A       2006-01-03    8.676239e+07
        2006-01-04    6.877919e+07
        2006-01-05    8.134003e+07
        2006-01-06    1.039317e+08
        2006-01-09    6.884032e+07
dtype: float64

In [13]:
top500 = (dv.groupby('date')
          .rank(ascending=False)
          
          # Drop stocks with less than 8 years of data
          .unstack('ticker')
          .dropna(thresh=8*252, axis='columns')
          
          # Average daily rank
          .mean()
          
          # Select the smallest rank b'coz ranking was done in descending (high --> low) order
          .nsmallest(500))

In [14]:
to_drop = data.index.unique('ticker').difference(top500.index)
len(to_drop)

2596

In [15]:
data = data.drop(index=to_drop, level='ticker')

data.info()
data.head()

print(f"# Tickers: {len(data.index.unique('ticker')):,.0f} | # Dates: {len(data.index.unique('date')):,.0f}")

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1366287 entries, ('A', Timestamp('2006-01-03 00:00:00')) to ('ZMH', Timestamp('2015-06-26 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   open    1366287 non-null  float64
 1   high    1366286 non-null  float64
 2   low     1366286 non-null  float64
 3   close   1366287 non-null  float64
 4   volume  1366287 non-null  float64
dtypes: float64(5)
memory usage: 57.4+ MB


open       high        low      close     volume
ticker date                                                             
A      2006-01-03  22.786839  22.909642  22.387796  22.855063  3796200.0
       2006-01-04  22.889175  23.080202  22.766372  22.916465  3001300.0
       2006-01-05  22.820951  23.516836  22.820951  23.516836  3458800.0
       2006-01-06  23.537304  23.735153  23.257585  23.639640  4396500.0
       2006-01-09  23.639640  23.741976  23.482724  23.571416  2920500.0

# Tickers: 500 | # Dates: 2,769


## Step 4

Now we’re ready to compute financial features. The Alpha Factory Library listed among the resources below illustrates how to compute a broad range of those using pandas and TA-Lib. We will list a few examples; feel free to explore and evaluate the various TA-Lib indicators.
- Compute historical returns for various time ranges such as 1, 3, 5, 10, and 21 trading days, as well as longer periods like 2, 3, 6, and 12 months.
- Use TA-Lib’s Bollinger Band indicator to create features that anticipate mean-reversion.
- Select some indicators from TA-Lib’s momentum indicators family, such as:
    - the Average Directional Movement Index (ADX),
    - the Moving Average Convergence Divergence (MACD),
    - the Relative Strength Index (RSI),
    - the Balance of Power (BOP) indicator, or
    - the Money Flow Index (MFI).
- Compute TA-Lib volume indicators like On Balance Volume (OBV) or the Chaikin A/D Oscillator (ADOSC).
- Create volatility metrics such as the Normalized Average True Range (NATR).
- Compute rolling factor betas using the five Fama-French risk factors for different rolling windows of 3 and 12 months (see Resources below).
- Compute the outcome variable that we will aim to predict, namely the 1-day forward returns.

In [16]:
by_ticker = data.groupby(level='ticker')

#### Historical returns

In [17]:
T = [1, 3, 5, 10, 21, 42, 63, 126, 252]

for t in T:
    data[f'ret_{t:02}'] = by_ticker.close.pct_change(t)

#### Forward returns

The outcome variable that we will aim to predict.

In [18]:
data['ret_fwd'] = by_ticker.ret_01.shift(-1)
data = data.dropna(subset=['ret_fwd'])

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1365787 entries, ('A', Timestamp('2006-01-03 00:00:00')) to ('ZMH', Timestamp('2015-06-25 00:00:00'))
Data columns (total 15 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   open     1365787 non-null  float64
 1   high     1365787 non-null  float64
 2   low      1365787 non-null  float64
 3   close    1365787 non-null  float64
 4   volume   1365787 non-null  float64
 5   ret_01   1365287 non-null  float64
 6   ret_03   1364287 non-null  float64
 7   ret_05   1363287 non-null  float64
 8   ret_10   1360787 non-null  float64
 9   ret_21   1355287 non-null  float64
 10  ret_42   1344787 non-null  float64
 11  ret_63   1334287 non-null  float64
 12  ret_126  1302787 non-null  float64
 13  ret_252  1239787 non-null  float64
 14  ret_fwd  1365787 non-null  float64
dtypes: float64(15)
memory usage: 161.6+ MB


#### Bollinger Bands

In [20]:
def compute_bb_indicators(close, timeperiod=20, matype=0):
    high, mid, low = talib.BBANDS(close, 
                                  timeperiod=20,
                                  matype=matype)
    bb_up = high / close - 1
    bb_low = low / close - 1
    squeeze = (high - low) / close
    return pd.DataFrame({'BB_UP': bb_up, 
                         'BB_LOW': bb_low, 
                         'BB_SQUEEZE': squeeze}, 
                        index=close.index)

In [21]:
data = (data.join(data
                  .groupby(level='ticker')
                  .close
                  .apply(compute_bb_indicators)))

#### Average Directional Movement Index (ADX)

In [22]:
def compute_adx(df, timeperiod=14):
    return talib.ADX(df.high,
                     df.low,
                     df.close,
                     timeperiod=timeperiod)

In [23]:
data['ADX'] = (data
               .groupby(level='ticker', group_keys=False)
               .apply(compute_adx))

#### Moving Average Convergence Divergence (MACD)

In [24]:
def compute_macd(close, fastperiod=12, slowperiod=26, signalperiod=9):
    macd, macdsignal, macdhist = talib.MACD(close,
                                            fastperiod=fastperiod,
                                            slowperiod=slowperiod,
                                            signalperiod=signalperiod)
    return pd.DataFrame({'MACD': macd,
                         'MACD_SIGNAL': macdsignal,
                         'MACD_HIST': macdhist},
                        index=close.index)

In [25]:
data = (data.join(data
                  .groupby(level='ticker')
                  .close
                  .apply(compute_macd)))

#### Relative Strength Index (RSI)

In [26]:
by_ticker = data.groupby(level='ticker', group_keys=False)

In [27]:
data['RSI'] = (by_ticker
               .apply(lambda x: talib.RSI(x.close,
                                          timeperiod=14)))

#### Balance of Power (BOP) indicator

In [28]:
data['BOP'] = (by_ticker
               .apply(lambda x: talib.BOP(x.open,
                                          x.high,
                                          x.low,
                                          x.close)))

#### Money Flow Index (MFI)

In [29]:
data['MFI'] = (by_ticker
               .apply(lambda x: talib.MFI(x.high,
                                          x.low,
                                          x.close,
                                          x.volume,
                                          timeperiod=14)))

#### On Balance Volume (OBV)

In [30]:
data['OBV'] = by_ticker.apply(lambda x: talib.OBV(x.close,
                                                  x.volume)/x.expanding().volume.mean())

#### Chaikin A/D Oscillator (ADOSC)

In [31]:
data['ADOSC'] = by_ticker.apply(lambda x: talib.ADOSC(x.high,
                                                      x.low,
                                                      x.close,
                                                      x.volume,
                                                      fastperiod=3,
                                                      slowperiod=10) / x.rolling(14).volume.mean())

#### Normalized Average True Range (NATR)

Compute normalized version of ATR using rolling mean of price.

In [32]:
data['ATR'] = by_ticker.apply(lambda x: talib.ATR(x.high,
                                                  x.low,
                                                  x.close,
                                                  timeperiod=14)/x.rolling(14).close.mean())

#### Rolling factor betas using the five Fama-French risk factors

In [33]:
factor_data = (pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3_daily',
                                       start='1-1-1998')[0].rename(columns={'Mkt-RF': 'MARKET'}))
factor_data.index.names = ['date']

factor_data.head()

,MARKET,SMB,HML,RMW,CMA,RF
date,,,,,,
1998-01-02,0.22,-0.12,-0.49,0.20,-0.14,0.021
1998-01-05,0.22,0.06,-0.29,0.03,-0.01,0.021
1998-01-06,-1.04,0.24,0.20,-0.38,0.14,0.021
1998-01-07,-0.38,-0.38,-0.22,-0.12,0.33,0.021
1998-01-08,-0.82,0.18,-0.09,0.05,0.29,0.021


In [34]:
factors = factor_data.columns[:-1]
factors

Index(['MARKET', 'SMB', 'HML', 'RMW', 'CMA'], dtype='object')

In [35]:
ret = 'ret_01'
windows = [63, 252]

for window in windows:
    print(window)
    
    betas = []
    for ticker, df in data.groupby('ticker', group_keys=False):
        model_data = df[[ret]].merge(factor_data, on='date').dropna()
        model_data[ret] -= model_data.RF

        rolling_ols = RollingOLS(endog=model_data[ret], 
                                 exog=sm.add_constant(model_data[factors]), window=window)
        factor_model = rolling_ols.fit(params_only=True).params.rename(columns={'const':'ALPHA'})
        result = factor_model.assign(ticker=ticker).set_index('ticker', append=True).swaplevel()
        betas.append(result)
    
    betas = pd.concat(betas).rename(columns=lambda x: f'{x}_{window:02}')
    data = data.join(betas)

63
252


#### Time Period Flags

In [36]:
dates = data.index.get_level_values('date')
dates[:5]

DatetimeIndex(['2006-01-03', '2006-01-04', '2006-01-05', '2006-01-06',
               '2006-01-09'],
              dtype='datetime64[ns]', name='date', freq=None)

In [37]:
data['month'] = dates.month.values.astype(np.uint8) - 1
data['weekday'] = dates.weekday.values.astype(np.uint8)

## Persist the results

In [38]:
data = (data
        .drop(columns=['open', 'high', 'low', 'close', 'volume'])
        .replace((np.inf, -np.inf), np.nan))

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1365787 entries, ('A', Timestamp('2006-01-03 00:00:00')) to ('ZMH', Timestamp('2015-06-25 00:00:00'))
Data columns (total 37 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   ret_01       1365287 non-null  float64
 1   ret_03       1364287 non-null  float64
 2   ret_05       1363287 non-null  float64
 3   ret_10       1360787 non-null  float64
 4   ret_21       1355287 non-null  float64
 5   ret_42       1344787 non-null  float64
 6   ret_63       1334287 non-null  float64
 7   ret_126      1302787 non-null  float64
 8   ret_252      1239787 non-null  float64
 9   ret_fwd      1365787 non-null  float64
 10  BB_UP        1356287 non-null  float64
 11  BB_LOW       1356287 non-null  float64
 12  BB_SQUEEZE   1356287 non-null  float64
 13  ADX          1352287 non-null  float64
 14  MACD         1349287 non-null  float64
 15  MACD_SIGNAL  1349287 non-null  float64
 16  MACD_HIST    13492

In [40]:
data.isnull().sum()

ret_01            500
ret_03           1500
ret_05           2500
ret_10           5000
ret_21          10500
ret_42          21000
ret_63          31500
ret_126         63000
ret_252        126000
ret_fwd             0
BB_UP            9500
BB_LOW           9500
BB_SQUEEZE       9500
ADX             13500
MACD            16500
MACD_SIGNAL     16500
MACD_HIST       16500
RSI              7000
BOP                 0
MFI              7000
OBV                 1
ADOSC            6516
ATR              7000
ALPHA_63        31500
MARKET_63       31500
SMB_63          31500
HML_63          31500
RMW_63          31500
CMA_63          31500
ALPHA_252      126000
MARKET_252     126000
SMB_252        126000
HML_252        126000
RMW_252        126000
CMA_252        126000
month               0
weekday             0
dtype: int64

In [42]:
data.to_hdf('us_stocks.h5', key='model_data', mode='a')